# Linear Regression on PYNQ-Z1
### n_features = 32, n_outputs=10

In [1]:
import numpy as np
from sklearn import datasets
from pynq_sklearn.linear_model import PynqLinearRegression
import time

# Generate dataset of Ints
X, y = datasets.make_regression(n_samples=5000, n_features=32, n_targets=10, random_state=0, noise=4.0,
                       bias=100.0)

In [2]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1000, random_state=42)
X_test = np.abs((X_test*(2**20)).astype("int32"))
y_test = np.abs((y_test*(2**20)).astype("int32"))

#### 1. Fit a PYNQ Linear Regression Model
(**Note: this notebook will only work with the Boston dataset (or datasets with the same shape). This is because our HW accelerator has been hardcoded)

In [3]:
lin = PynqLinearRegression(fit_intercept=True, hw_accel=True)
lin.fit(X_train, y_train)

#### 2. With hw_accel=True, deploy prediction on the FPGA 

In [4]:
y_pred = lin.predict(X_test)

Offloading predict to FPGA ...


#### 3. Alternatively, deploy in SW

In [5]:
lin.hw_accel=False
y_pred_sw = lin.predict(X_test)

#### 4. Verify equivalence

In [6]:
import numpy as np
np.allclose(y_pred, y_pred_sw, )

True

#### 5. Performance
Bad performance observed because of sw-to-hw processing in lin.predict()

In [7]:
import timeit
number=20

lin.hw_accel=True
def hwresp():
    lin.predict(X_test)

hw_time = timeit.timeit(hwresp,number=number)

lin.hw_accel=False
def swresp():
    lin.predict(X_test)

sw_time = timeit.timeit(swresp,number=number)

print("Time taken by sklearn", number,"times",sw_time)
print("Time taken by sklearn+fpga", number,"times",hw_time)
print("HW Speedup = %dx"%(sw_time/hw_time))

Offloading predict to FPGA ...
Offloading predict to FPGA ...
Offloading predict to FPGA ...
Offloading predict to FPGA ...
Offloading predict to FPGA ...
Offloading predict to FPGA ...
Offloading predict to FPGA ...
Offloading predict to FPGA ...
Offloading predict to FPGA ...
Offloading predict to FPGA ...
Offloading predict to FPGA ...
Offloading predict to FPGA ...
Offloading predict to FPGA ...
Offloading predict to FPGA ...
Offloading predict to FPGA ...
Offloading predict to FPGA ...
Offloading predict to FPGA ...
Offloading predict to FPGA ...
Offloading predict to FPGA ...
Offloading predict to FPGA ...
Time taken by sklearn 20 times 0.08533485499992821
Time taken by sklearn+fpga 20 times 2.6650082829999064
HW Speedup = 0x
